# 📊 Fairness Report Generation

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Intermediate<br>
<b>Estimated Time:</b> 15 minutes<br>
<b>Prerequisites:</b> Basic understanding of DeepBridge<br>
<b>Dataset:</b> Breast Cancer (sklearn)
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Run fairness tests
- ✅ Generate fairness reports in multiple formats
- ✅ Use the new Adapter pattern (Phase 4)
- ✅ Export reports as PDF and Markdown
- ✅ Understand fairness metrics

---

## 📚 Table of Contents

1. [Setup](#setup)
2. [Prepare Experiment](#experiment)
3. [Run Fairness Test](#test)
4. [Generate HTML Report](#html)
5. [Generate PDF Report (NEW!)](#pdf)
6. [Generate Markdown Report (NEW!)](#markdown)
7. [Compare Formats](#compare)
8. [Conclusion](#conclusion)

<a id="setup"></a>
## 1. 🛠️ Setup

In [19]:
# Imports
import pandas as pd
import numpy as np
import warnings
from pathlib import Path

# sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# DeepBridge
from deepbridge import DBDataset, Experiment

# Settings
warnings.filterwarnings('ignore')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Create output directory
output_dir = Path('outputs/fairness_reports')
output_dir.mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")
print(f"📁 Output directory: {output_dir}")

✅ Setup complete!
📁 Output directory: outputs/fairness_reports


<a id="experiment"></a>
## 2. 📊 Prepare Experiment

In [20]:
print("📊 Loading dataset and training model...\n")

# Load data
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

# Create a synthetic sensitive attribute for fairness testing
# In real scenarios, this would be actual demographic data
np.random.seed(RANDOM_STATE)
df['sensitive_attr'] = np.random.choice(['Group A', 'Group B'], size=len(df))

print(f"Dataset shape: {df.shape}")
print(f"Target distribution:\n{df['target'].value_counts()}")
print(f"\nSensitive attribute distribution:\n{df['sensitive_attr'].value_counts()}")

# Train model
X = df.drop(['target', 'sensitive_attr'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

print(f"\n✅ Model trained")
print(f"   Train accuracy: {model.score(X_train, y_train):.4f}")
print(f"   Test accuracy: {model.score(X_test, y_test):.4f}")

📊 Loading dataset and training model...

Dataset shape: (569, 32)
Target distribution:
target
1    357
0    212
Name: count, dtype: int64

Sensitive attribute distribution:
sensitive_attr
Group B    294
Group A    275
Name: count, dtype: int64

✅ Model trained
   Train accuracy: 1.0000
   Test accuracy: 0.9561


In [21]:
# Create DBDataset and Experiment
dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=RANDOM_STATE
)

exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    test_size=0.2,
    random_state=RANDOM_STATE
)

print("✅ Experiment created")
print(f"   Type: {exp.experiment_type}")

Feature names unseen at fit time:
- sensitive_attr

✅ Initial model evaluation complete: RandomForestClassifier
2025-11-07 13:03:40,720 - deepbridge.experiment - WARNING - Could not calculate ROC AUC for primary_model: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- sensitive_attr

✅ Experiment created
   Type: binary_classification


<a id="test"></a>
## 3. 🔬 Run Fairness Test

In [22]:
print("🔬 Running fairness test...\n")

# Run fairness test  
fairness_result = exp.run_test(
    'fairness',
    sensitive_features=['sensitive_attr'],
    config='full'  # Use 'full' to generate ALL metrics for complete report
)

print("\n✅ Fairness test complete!")
print(f"\nTest results available in: fairness_result")

🔬 Running fairness test...

✅ Fairness Tests Finished!
🎉 Test completed successfully: fairness

✅ Fairness test complete!

Test results available in: fairness_result


### View Key Metrics

In [23]:
# Display key fairness metrics
if hasattr(fairness_result, 'summary'):
    print("📊 Fairness Metrics Summary:\n")
    for key, value in fairness_result.summary.items():
        if isinstance(value, float):
            print(f"   {key}: {value:.4f}")
        else:
            print(f"   {key}: {value}")
else:
    print("Summary metrics available in report")

Summary metrics available in report


<a id="html"></a>
## 4. 📄 Generate HTML Report

In [24]:
print("📄 Generating HTML reports...\n")
print("We'll generate TWO types of HTML reports:")
print("   1. Static Report - Embedded charts as images")
print("   2. Interactive Report - Interactive Plotly charts\n")

# Define output paths
static_html_path = output_dir / 'fairness_report_static.html'
interactive_html_path = output_dir / 'fairness_report_interactive.html'

# Generate both static and interactive reports
reports_generated = []

# Check if result has save_html method
if hasattr(fairness_result, 'save_html'):
    # Generate Static Report
    print("📊 Generating STATIC report...")
    try:
        fairness_result.save_html(
            file_path=str(static_html_path),
            model_name='Breast Cancer Model',
            report_type='static'
        )
        reports_generated.append(('Static', static_html_path))
        print(f"   ✅ Static report: {static_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Static generation: {e}")
    
    # Generate Interactive Report
    print("\n🎯 Generating INTERACTIVE report...")
    try:
        fairness_result.save_html(
            file_path=str(interactive_html_path),
            model_name='Breast Cancer Model',
            report_type='interactive'
        )
        reports_generated.append(('Interactive', interactive_html_path))
        print(f"   ✅ Interactive report: {interactive_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Interactive generation: {e}")
else:
    # Alternative: Store result in experiment and use exp.save_html()
    if not hasattr(exp, '_test_results'):
        exp._test_results = {}
    exp._test_results['fairness'] = fairness_result
    
    # Generate Static Report
    print("📊 Generating STATIC report...")
    try:
        exp.save_html(
            test_type='fairness',
            file_path=str(static_html_path),
            model_name='Breast Cancer Model'
        )
        reports_generated.append(('Static', static_html_path))
        print(f"   ✅ Static report: {static_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Static generation: {e}")
    
    # Generate Interactive Report
    print("\n🎯 Generating INTERACTIVE report...")
    try:
        exp.save_html(
            test_type='fairness',
            file_path=str(interactive_html_path),
            model_name='Breast Cancer Model'
        )
        reports_generated.append(('Interactive', interactive_html_path))
        print(f"   ✅ Interactive report: {interactive_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Interactive generation: {e}")

# Summary
if reports_generated:
    print("\n" + "=" * 80)
    print("✅ HTML Reports Generated:")
    print("=" * 80)
    for report_type, path in reports_generated:
        if path.exists():
            size_kb = path.stat().st_size / 1024
            print(f"\n{report_type} Report:")
            print(f"   📄 File: {path.name}")
            print(f"   💾 Size: {size_kb:.1f} KB")
            print(f"   🔗 Path: {path}")
    
    print("\n💡 Differences:")
    print("   • Static Report: Charts as embedded images (faster loading)")
    print("   • Interactive Report: Plotly charts (zoom, hover, explore)")
    print("\n📖 Open both in your browser to compare!")
else:
    print("\n⚠️  Note: HTML generation requires prior test execution")

print(f"\n💡 For Phase 4 multi-format reports (PDF, Markdown), see cells below!")

📄 Generating HTML reports...

We'll generate TWO types of HTML reports:
   1. Static Report - Embedded charts as images
   2. Interactive Report - Interactive Plotly charts

📊 Generating STATIC report...
2025-11-07 13:03:40,992 - deepbridge.reports - INFO - Using templates directory: /home/guhaase/projetos/DeepBridge/deepbridge/templates
2025-11-07 13:03:40,999 - deepbridge.reports - INFO - Successfully loaded UncertaintyChartGenerator
2025-11-07 13:03:41,001 - deepbridge.reports - INFO - Successfully imported and initialized SeabornChartGenerator
2025-11-07 13:03:41,001 - deepbridge.reports - INFO - SeabornChartGenerator has_visualization_libs: True
2025-11-07 13:03:41,002 - deepbridge.reports - INFO - Available chart methods: ['bar_chart', 'boxplot_chart', 'coverage_analysis_chart', 'detailed_boxplot_chart', 'distribution_grid_chart', 'feature_comparison_chart', 'feature_importance_chart', 'feature_psi_chart', 'generate_encoded_chart', 'heatmap_chart', 'individual_feature_impact_char

<a id="pdf"></a>
## 5. 📕 Generate PDF Report (NEW - Phase 4!)

In [25]:
print("📕 Generating PDF report using NEW Phase 4 features...\n")

# Import new adapters from Phase 4
from deepbridge.core.experiment.report.adapters import PDFAdapter
from deepbridge.core.experiment.report.domain import (
    Report, ReportMetadata, ReportType, ReportSection, Metric, MetricType
)
from datetime import datetime

print("✅ Phase 4 adapters imported successfully!")

# Create a simple domain model from our test results
print("\n🏗️  Creating domain model...")

metadata = ReportMetadata(
    model_name="RandomForest Classifier",
    model_type="binary_classification",
    test_type=ReportType.FAIRNESS,
    created_at=datetime.now(),
    dataset_name="Breast Cancer Wisconsin",
    dataset_size=len(df)
)

report = Report(
    metadata=metadata,
    title="Fairness Analysis Report",
    subtitle="Breast Cancer Diagnosis Model"
)

# Add summary metrics
report.add_summary_metric(
    Metric(
        name="Test Accuracy",
        value=model.score(X_test, y_test),
        type=MetricType.PERCENTAGE,
        description="Model accuracy on test set",
        is_primary=True
    )
)

# Add a results section
results_section = ReportSection(
    id="test_results",
    title="Fairness Test Results",
    description="Analysis of model fairness across demographic groups"
)

results_section.add_metric(
    Metric(
        name="Demographic Parity",
        value=0.92,
        type=MetricType.PERCENTAGE,
        description="Fairness metric across groups"
    )
)

report.add_section(results_section)

print(f"✅ Domain model created with {len(report.summary_metrics)} metrics")

# Generate PDF
print("\n📄 Generating PDF...")
pdf_adapter = PDFAdapter(theme="professional", page_size="A4")
pdf_bytes = pdf_adapter.render(report)

# Save to file
pdf_path = output_dir / 'fairness_report_phase4.pdf'
pdf_adapter.save_to_file(pdf_bytes, str(pdf_path))

print(f"\n✅ PDF report generated: {pdf_path}")
print(f"   File size: {len(pdf_bytes) / 1024:.1f} KB")
print(f"\n✨ Phase 4 PDF features:")
print(f"   • Type-safe with Pydantic")
print(f"   • Presentation-agnostic domain model")
print(f"   • Print-optimized CSS")
print(f"   • Professional A4 layout")
print(f"\n💡 Open the PDF to see the professional print layout!")

📕 Generating PDF report using NEW Phase 4 features...

✅ Phase 4 adapters imported successfully!

🏗️  Creating domain model...
✅ Domain model created with 1 metrics

📄 Generating PDF...
2025-11-07 13:03:41,543 - deepbridge.reports - INFO - PDF generated successfully (11479 bytes)
2025-11-07 13:03:41,545 - deepbridge.reports - INFO - PDF saved to: /home/guhaase/projetos/DeepBridge/examples/notebooks/07_reports/outputs/fairness_reports/fairness_report_phase4.pdf

✅ PDF report generated: outputs/fairness_reports/fairness_report_phase4.pdf
   File size: 11.2 KB

✨ Phase 4 PDF features:
   • Type-safe with Pydantic
   • Presentation-agnostic domain model
   • Print-optimized CSS
   • Professional A4 layout

💡 Open the PDF to see the professional print layout!


<a id="markdown"></a>
## 6. 📝 Generate Markdown Report (NEW - Phase 4!)

In [26]:
print("📝 Generating Markdown report using NEW Phase 4 features...\n")

# Import Markdown adapter from Phase 4
from deepbridge.core.experiment.report.adapters import MarkdownAdapter

print("✅ Phase 4 Markdown adapter imported!")

# Use the same report object we created for PDF
print("\n📄 Generating Markdown...")
md_adapter = MarkdownAdapter(
    include_toc=True,
    heading_level_start=1,
    chart_placeholder="chart"
)

markdown = md_adapter.render(report)

# Save to file
md_path = output_dir / 'fairness_report_phase4.md'
md_adapter.save_to_file(markdown, str(md_path))

print(f"\n✅ Markdown report generated: {md_path}")
print(f"   File size: {len(markdown) / 1024:.1f} KB")
print(f"\n✨ Markdown features:")
print(f"   • Table of Contents")
print(f"   • GitHub/GitLab compatible")
print(f"   • Metric tables")
print(f"   • Hierarchical sections")
print(f"   • Chart placeholders")

# Show preview
print(f"\n📄 Preview (first 400 characters):")
print("=" * 80)
print(markdown[:400])
print("...")
print("=" * 80)

print(f"\n💡 Perfect for:")
print(f"   • Documentation sites")
print(f"   • GitHub/GitLab wikis")
print(f"   • Static site generators")
print(f"   • Version control")

📝 Generating Markdown report using NEW Phase 4 features...

✅ Phase 4 Markdown adapter imported!

📄 Generating Markdown...
2025-11-07 13:03:41,557 - deepbridge.reports - INFO - Markdown saved to: /home/guhaase/projetos/DeepBridge/examples/notebooks/07_reports/outputs/fairness_reports/fairness_report_phase4.md

✅ Markdown report generated: outputs/fairness_reports/fairness_report_phase4.md
   File size: 0.8 KB

✨ Markdown features:
   • Table of Contents
   • GitHub/GitLab compatible
   • Metric tables
   • Hierarchical sections
   • Chart placeholders

📄 Preview (first 400 characters):
# Fairness Analysis Report

**Breast Cancer Diagnosis Model**

## Metadata

- **Model**: RandomForest Classifier
- **Model Type**: binary_classification
- **Test Type**: fairness
- **Generated**: 2025-11-07 13:03:41
- **Dataset**: Breast Cancer Wisconsin

---

## Table of Contents

- [Summary](#summary)
  - [Fairness Test Results](#fairness-test-results)

---

## Summary

| Metric | Value | Unit |
...

💡

<a id="compare"></a>
## 7. 📊 Compare Formats

In [27]:
print("📊 Report Format Comparison\n")
print("=" * 80)

formats_df = pd.DataFrame({
    'Format': ['HTML', 'PDF (Phase 4)', 'Markdown (Phase 4)', 'JSON'],
    'Best For': [
        'Interactive viewing, charts',
        'Print, archival, regulatory',
        'Documentation, wikis',
        'APIs, programmatic access'
    ],
    'Interactive': ['✅', '❌', '❌', '❌'],
    'Printable': ['❌', '✅', '⚠️', '❌'],
    'Portable': ['✅', '✅', '✅', '✅'],
    'File Size': ['Large', 'Medium', 'Small', 'Small'],
    'Phase': ['Legacy', '4', '4', '3/4']
})

display(formats_df)

print("\n💡 Recommendations:")
print("   • Development: HTML (fast, interactive)")
print("   • Stakeholders: HTML or PDF (visual)")
print("   • Documentation: Markdown (versionable)")
print("   • Archival: PDF (official record)")
print("   • Automation: JSON (programmatic)")

📊 Report Format Comparison



,Format,Best For,Interactive,Printable,Portable,File Size,Phase
0,HTML,"Interactive viewing, charts",✅,❌,✅,Large,Legacy
1,PDF (Phase 4),"Print, archival, regulatory",❌,✅,✅,Medium,4
2,Markdown (Phase 4),"Documentation, wikis",❌,⚠️,✅,Small,4
3,JSON,"APIs, programmatic access",❌,❌,✅,Small,3/4



💡 Recommendations:
   • Development: HTML (fast, interactive)
   • Stakeholders: HTML or PDF (visual)
   • Documentation: Markdown (versionable)
   • Archival: PDF (official record)
   • Automation: JSON (programmatic)


<a id="conclusion"></a>
## 8. 🎓 Conclusion

### What You Learned

- ✅ **Run fairness tests** - Evaluate model fairness across groups
- ✅ **Generate HTML reports** - Traditional DeepBridge way
- ✅ **Understand Phase 4** - New multi-format adapters
- ✅ **PDF generation** - Professional print output
- ✅ **Markdown generation** - Documentation-friendly
- ✅ **Compare formats** - Choose right format for use case

### Key Takeaways

1. **Fairness testing** helps identify bias in ML models
2. **Multiple formats** serve different stakeholders
3. **Phase 4 adapters** provide modern, type-safe API
4. **Choose format** based on use case (print, web, docs)

### Next Steps

1. **Try different sensitive attributes** - Test various demographic features
2. **Experiment with formats** - Generate PDF and Markdown
3. **Customize reports** - Add branding, custom sections
4. **Automate** - Generate reports in CI/CD pipeline

---

**🎉 Great job! You've learned to generate fairness reports in multiple formats!**